In [1]:
import os

In [2]:
%pwd

'/Users/natali/Code/Playground/image-classification-kidney/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/natali/Code/Playground/image-classification-kidney'

In [26]:
path = '/Users/natali/Code/Playground/image-classification-kidney/artifacts/data_ingestion'

In [27]:
import os

for path, subfolders, filenames in os.walk(path):
    print(path, subfolders, filenames)

/Users/natali/Code/Playground/image-classification-kidney/artifacts/data_ingestion ['CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'] ['kidneyData.csv', 'data.zip']
/Users/natali/Code/Playground/image-classification-kidney/artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone ['CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'] []
/Users/natali/Code/Playground/image-classification-kidney/artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone ['Tumor', 'Cyst', 'Stone', 'Normal'] []
/Users/natali/Code/Playground/image-classification-kidney/artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Tumor [] ['Tumor- (2252).jpg', 'Tumor- (1887).jpg', 'Tumor- (1184).jpg', 'Tumor- (90).jpg', 'Tumor- (508).jpg', 'Tumor- (1491).jpg', 'Tumor- (158).jpg', 'Tumor- (1868).jpg', 'Tumor- (271).jpg', 'Tumor- (1242).jpg', 'Tumor- (621).jpg', 'Tumor- (1612).jpg', 'Tumor- (2194).jpg', 'Tumor- (1307).

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [7]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2023-11-12 08:46:28.571169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [14]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [15]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [28]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2023-11-12 09:35:28,215: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-12 09:35:28,220: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-12 09:35:28,221: INFO: common: created directory at: artifacts]
[2023-11-12 09:35:28,223: INFO: common: created directory at: artifacts/training]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/kidney-ct-scan-image'